In [2]:
%matplotlib inline

import pandas as pd
import numpy as np

import glob
import re

import matplotlib.pyplot as plt

from collections import Counter

### Read in the Data and a bit of Clean Up

In [3]:
###########################################
# create a function to combine data files #
###########################################

def frame_masher():
    # find all of the query files
    numbers = [1,4,5,6,7,11,12,15] # combine these datafiles
    query_files = []
    for num in numbers:
        file ='data/tea_party/tea_party{}.csv'.format(num)
        query_files.append(file)
    
    
    # empty list to add data frames to
    frame_list = []
    
    # iterate through the files to create one combined date frame
    for frame in query_files:
        df = pd.read_csv(frame)
        frame_list.append(df)
    joined = pd.concat(frame_list).reset_index()
    final = joined.drop('index',axis = 1)
    return final

In [4]:
# create the combined data frame
df = frame_masher()

# little bit of clean up
df = df.loc[df['job_id'] == '2572']

/usr/lib/python3.4/site-packages/IPython/core/interactiveshell.py:2825: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/usr/lib/python3.4/site-packages/IPython/core/interactiveshell.py:2825: DtypeWarning: Columns (8,9,12) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/usr/lib/python3.4/site-packages/IPython/core/interactiveshell.py:2825: DtypeWarning: Columns (0,3,4,9,12) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/usr/lib/python3.4/site-packages/IPython/core/interactiveshell.py:2825: DtypeWarning: Columns (0,1,2,3,4,5,6,9,11,12,14) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/usr/lib/python3.4/site-packages/IPython/core/interactiveshell.py:2825: DtypeWarning: Columns (3,4,5,6,9,11,12,14) have mixed types. Speci

In [6]:
df['created_at']=pd.to_datetime(df['created_at'],format='%Y-%m-%d %H:%M:%S')
df['Month'] = df['created_at'].dt.month
df['Year'] = df['created_at'].dt.year
df['Day'] = df['created_at'].dt.day

### Clean up tweets

In [16]:
# Pre-process tweets

def processTweet(tweet):
    # process the tweets
    tweet = str(tweet)
    #Convert to lower case
    tweet = tweet.lower()
    #Convert www.* or https?://* to URL
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet)
    #Convert @username to AT_USER
    tweet = re.sub('@[^\s]+','AT_USER',tweet)
    #Remove additional white spaces
    tweet = re.sub('[\s]+', ' ', tweet)
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    #trim
    tweet = tweet.strip('\'"')
    return tweet
#end

In [19]:
# create a function to iterate over an entire column of tweets and apply the processTweet function
def tweetCleaner(column):
    tweets = []
    for text in column:
        cleaned = processTweet(text)
        tweets.append(cleaned)
    return tweets

In [21]:
# write result to the data frame
df['tweet']= tweetCleaner(df['text'])

### Aggregate by how many terms per month

In [42]:
# here is a function that will count the occurances of a column per month and return a data frame

def dataAggregator(df,column):
    new = df.groupby(['Year','Month','Day'])[column].agg({'no':'count'})
    new.reset_index(level=[0,1,2], inplace=True)
    new['datetime'] = pd.to_datetime((new.Year*10000+new.Month*100+new.Day).apply(str),format='%Y%m%d')
    return new

In [43]:
dataAggregator(df,'job_id')

,Year,Month,Day,no,datetime
0,2014,8,5,5819,2014-08-05
1,2014,8,6,13722,2014-08-06
2,2014,8,7,10057,2014-08-07
3,2014,8,8,9876,2014-08-08
4,2014,8,9,7163,2014-08-09
5,2014,8,10,6632,2014-08-10
6,2014,8,11,8693,2014-08-11
7,2014,8,12,8567,2014-08-12
8,2014,8,13,9300,2014-08-13
9,2014,8,14,15388,2014-08-14


In [37]:
obama =df[df["tweet"].str.contains("obama")]
stupid =df[df["tweet"].str.contains("stupid")]

In [44]:
dataAggregator(stupid,'job_id')

,Year,Month,Day,no,datetime
0,2014,8,5,17,2014-08-05
1,2014,8,6,30,2014-08-06
2,2014,8,7,24,2014-08-07
3,2014,8,8,45,2014-08-08
4,2014,8,9,21,2014-08-09
5,2014,8,10,13,2014-08-10
6,2014,8,11,26,2014-08-11
7,2014,8,12,22,2014-08-12
8,2014,8,13,19,2014-08-13
9,2014,8,14,35,2014-08-14


In [45]:
dataAggregator(obama,'job_id')

,Year,Month,Day,no,datetime
0,2014,8,5,383,2014-08-05
1,2014,8,6,638,2014-08-06
2,2014,8,7,770,2014-08-07
3,2014,8,8,414,2014-08-08
4,2014,8,9,421,2014-08-09
5,2014,8,10,408,2014-08-10
6,2014,8,11,418,2014-08-11
7,2014,8,12,355,2014-08-12
8,2014,8,13,273,2014-08-13
9,2014,8,14,673,2014-08-14
